## Data preprocessing

### Loading packages

In [1]:
import pandas as pd
import numpy as np

### Loading data

In [2]:
df_uti = pd.read_csv('C:/Users/ADMINLINAPD/Documents/data/UTI_pos_1212.csv', delimiter = ';')

df_sex = pd.read_csv('C:/Users/ADMINLINAPD/Documents/data/sex_0512.csv', delimiter = ';')

df_age = pd.read_csv('C:/Users/ADMINLINAPD/Documents/data/age_0512.csv', delimiter = ';')

df_SFI = pd.read_csv('C:/Users/ADMINLINAPD/Documents/data/SFI_urination_0512.csv', delimiter = ';')

df_acute_days = pd.read_csv('C:/Users/ADMINLINAPD/Documents/data/acute_days_0512.csv', delimiter = ';')

df_FIM_total = pd.read_csv('C:/Users/ADMINLINAPD/Documents/data/FIM_total_0512.csv', delimiter = ';')

df_brain_injury_mod = pd.read_csv('C:/Users/ADMINLINAPD/Documents/data/brain_injury_modified_0512.csv', delimiter = ';')


df_SFI_psyk = pd.read_csv('C:/Users/ADMINLINAPD/Documents/data/SFI_psykosocial_group_0512.csv', delimiter = ';')

In [3]:
print(df_brain_injury_mod['injury_modified'].unique())


['Apopleksi' 'Traumatisk' 'SAH' 'Ingen' 'Infektioner' 'Encephalopati'
 'Blødning' 'Fejl' 'Tumor' 'Andet' 'Funktionel lidelse' 'Ukendt']


In [ ]:
df_FIM_total

In [ ]:
#Checking that it worked
print(len(df_sex['ID'].unique()))
print(len(df_sex))
print(len(df_age))
print(len(df_uti))
print(len(df_brain_injury_mod))

In [6]:
print(df_sex['sex'].unique())

df_sex = df_sex[df_sex['sex'].isin(['Kvinde', 'Mand'])]

#Replacing strings with 1 and 0 and rename df
df_female = df_sex.replace('Kvinde', 1)
df_female = df_female.replace('Mand', 0)

print(df_female['sex'].unique()) #checking 

['Kvinde' 'Mand' nan]
[1 0]


In [7]:
#Printing the ID's of patients with sex == nan
print(np.setdiff1d(df_age['ID'],df_female['ID']))


#Removing the patient without gender from the other dfs
df_age = df_age[df_age['ID'].isin(df_female['ID'])]
print(len(df_age)) #test
df_SFI = df_SFI[df_SFI['ID'].isin(df_female['ID'])]
print(len(df_SFI)) #test
df_acute_days = df_acute_days[df_acute_days['ID'].isin(df_female['ID'])]
df_FIM_total = df_FIM_total[df_FIM_total['ID'].isin(df_female['ID'])]
df_brain_injury_mod = df_brain_injury_mod[df_brain_injury_mod['ID'].isin(df_female['ID'])]


df_SFI_psyk = df_SFI_psyk[df_SFI_psyk['ID'].isin(df_female['ID'])]


[5392069]
1864
133086


In [ ]:
df_SFI

#Checking for dublicates, if numbers are the same, there are no dublicates
print(len(df_SFI)) #133.086
print(len(df_SFI.drop_duplicates()))  #133.086

df1 = df_SFI[df_SFI.isna().any(axis=1)]
print(df1)

df_SFI = df_SFI.dropna()

print(len(df_SFI)) #133.085, that is, one empty note was removed




#Checking for dublicates, if numbers are the same, there are no dublicates
print(len(df_SFI_psyk)) #133.086
print(len(df_SFI_psyk.drop_duplicates()))  #133.086

df1_psyk = df_SFI_psyk[df_SFI_psyk.isna().any(axis=1)]
print(df1_psyk)

df_SFI_psyk = df_SFI_psyk.dropna()

print(len(df_SFI_psyk)) #133.085, that is, one empty note was removed

In [9]:
#Are there any ID's in SFI, not in df_female?
print(df_SFI['ID'].isin(df_female['ID']).value_counts())
# No

#Are there any ID's in UTI, not in df_female?
print(df_uti['ID'].isin(df_female['ID']).value_counts())
# Yes, 180 (506 are in both). I.e. 180 patients with UTI do not have data in urination SFI, that is a lot, why is this number so large??
# Some are < 18

#Are there any ID's in df_female, not in UTI?
print(df_female['ID'].isin(df_uti['ID']).value_counts())
#Yes, 1358, these are all the patients that do not get a UTI

#Are there any ID's in SFI, not in df_female?
print(df_SFI_psyk['ID'].isin(df_female['ID']).value_counts())
# No



True    133085
Name: ID, dtype: int64
True     518
False    187
Name: ID, dtype: int64
False    1346
True      518
Name: ID, dtype: int64
True    111864
Name: ID, dtype: int64


In [ ]:
df_acute_days

In [11]:
#Merging to ensure that we only keep the UTI patients who also have free-text data
mergedStuff = pd.merge(df_age, df_uti, on=['ID'], how='left')
mergedStuff


print(len(mergedStuff['ID'].unique()))

print(mergedStuff['UTI'].unique()) #checking what the column contains

filtered_df = mergedStuff[mergedStuff['UTI'].notnull()] #deleting UTI neg from the df so I can make a new UTI df 
#filtered_df = filtered_df.replace(1.0, 1)
filtered_df['UTI'] = filtered_df['UTI'].astype('int') #For some reason merge changed the values to floats, therefore I change them back
print(filtered_df['UTI'].unique()) #checking that it worked

print(len(filtered_df))


1864
[ 1. nan]
[1]
518


C:\Users\ADMINLINAPD\AppData\Local\Temp\11\ipykernel_4600\1291876411.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['UTI'] = filtered_df['UTI'].astype('int') #For some reason merge changed the values to floats, therefore I change them back


In [ ]:
#Taking only relevant columns
df_uti = filtered_df[['ID', 'UTI', 'date']]

df_uti

#rename UTI to value (column-name needs to be value for timeseriesflattener)
df_uti = df_uti.rename(columns= {'UTI': 'value'})

df_uti

In [ ]:
df_admissions = pd.read_csv('C:/Users/ADMINLINAPD/Documents/data/admissions_0512.csv', delimiter = ';')

df_admissions 
df_admissions['admission_start']= pd.to_datetime(df_admissions['admission_start']).dt.date  #Last bit deletes time of day
df_admissions['admission_end']= pd.to_datetime(df_admissions['admission_end']).dt.date
df_admissions



In [ ]:

#Create column of all days each patient is admitted, based on admission_start and admission_end 
df_admissions2 = pd.concat([pd.DataFrame({'date': pd.date_range(row['admission_start'], row['admission_end'], freq='D'), 'ID': row['ID']})
           for i, row in df_admissions.iterrows()], ignore_index=True)

def first_last(df_admissions2):
     return df_admissions2.iloc[1:-1]

df_admissions2 = df_admissions2[df_admissions2.duplicated('ID', keep='last')] #removing last day of every group (day of discharge)
df_admissions2 = df_admissions2[df_admissions2.duplicated('ID', keep='first')] #removing first day of every group (day of admission)

sub = df_admissions2[df_admissions2['ID']== 8769379] #Checking if it worked on a random patient
sub

In [ ]:
#adding the wanted time of prediction 
df_admissions2['date'] = df_admissions2['date'] + pd.Timedelta(hours=8, minutes=00, seconds=00)
df_admissions2

In [16]:
# #Writing csvs of the dfs
# df_uti.to_csv("test_data2/df_uti.csv")
# df_SFI.to_csv("test_data2/df_SFI.csv")
# df_admissions2.to_csv("test_data2/df_admissions.csv")
# df_age.to_csv("test_data2/df_age.csv")
# df_female.to_csv("test_data2/df_female.csv")
# df_FIM_total.to_csv("test_data2/df_FIM_total.csv")
# df_acute_days.to_csv("test_data2/df_acute_days.csv")
# df_brain_injury_mod.to_csv("test_data2/df_brain_injury_mod.csv")

df_SFI_psyk.to_csv("test_data2/df_SFI_psyk.csv")

## Additional necessary cleaning

In [1]:
import pandas as pd
import numpy as np

#loading data
df_female = pd.read_csv('test_data2/df_female.csv', index_col=[0])
df_age = pd.read_csv('test_data2/df_age.csv', index_col=[0])
df_SFI = pd.read_csv('test_data2/df_SFI.csv', index_col=[0])
df_uti = pd.read_csv('test_data2/df_uti.csv', index_col=[0])
df_admissions = pd.read_csv('test_data2/df_admissions.csv', index_col=[0])
df_FIM_total = pd.read_csv('test_data2/df_FIM_total.csv', index_col=[0])
df_acute_days = pd.read_csv('test_data2/df_acute_days.csv', index_col=[0])
df_brain_injury = pd.read_csv('test_data2/df_brain_injury_mod.csv', index_col=[0])


#rename FIM_total to value (column-name needs to be value for timeseriesflattener)
df_FIM_total = df_FIM_total.rename(columns= {'FIM_total': 'value'})


#Loading resampled data and removing extra column
df_SFI_resampled = pd.read_csv('test_data2/df_SFI_resampled.csv', index_col=[0])
df_admissions_resampled = pd.read_csv('test_data2/df_prediction_times_resampled.csv', index_col=[0])

In [2]:
#Creating uti df with negative cases as well
merged = pd.merge(df_age, df_uti, on=['ID'], how='left')

merged = merged[['ID','value']]

#turning UVI-neg cases from nan to 0
where_are_NaNs = np.isnan(merged)
merged[where_are_NaNs] = 0

#checking
#print(merged)

#Changing from floats to int
merged['value'] = merged['value'].astype('int')

df_outcome = pd.merge(merged, df_uti, on=['ID', 'value'], how='left')

#print(df_outcome)


#Need a little preprocessing for FIM scores to be available for all dates in the admission. Always using the latest available FIM score for each patient
df_admissions_without_time = df_admissions.copy() 
df_admissions_without_time['date'] = pd.to_datetime(df_admissions_without_time['date']).dt.date
df_FIM_total['date'] = pd.to_datetime(df_FIM_total['date']).dt.date

df_merge = pd.merge(df_admissions_without_time, df_FIM_total, on=['ID', 'date'], how='left')

df_merge_filled = df_merge.copy() 
df_merge_filled.update(df_merge_filled.sort_values(["ID", "date"]).groupby("ID").ffill()) 

df_FIM_total = df_merge_filled

In [3]:
#including only the patients that are in the resampled df 
patients = df_SFI_resampled['ID'].unique()

df_female = df_female.loc[df_female['ID'].isin(patients)]
df_age = df_age.loc[df_age['ID'].isin(patients)]
df_uti = df_uti.loc[df_uti['ID'].isin(patients)]
df_admissions = df_admissions.loc[df_admissions['ID'].isin(patients)]
df_FIM_total = df_FIM_total.loc[df_FIM_total['ID'].isin(patients)]
df_acute_days = df_acute_days.loc[df_acute_days['ID'].isin(patients)]
df_brain_injury = df_brain_injury.loc[df_brain_injury['ID'].isin(patients)]
df_outcome= df_outcome.loc[df_outcome['ID'].isin(patients)]
df_SFI=df_SFI.loc[df_SFI['ID'].isin(patients)]

df_admissions_resampled = df_admissions_resampled.loc[df_admissions_resampled['ID'].isin(patients)] #even though it contained only one obs pr person, it still contained all patients, even those without notes in the days of interest


In [4]:
#Writing csvs of the dfs
df_uti.to_csv("data/preprocessed_data/df_uti.csv")
df_SFI.to_csv("data/preprocessed_data/df_SFI.csv")
df_admissions.to_csv("data/preprocessed_data/df_admissions.csv")
df_age.to_csv("data/preprocessed_data/df_age.csv")
df_female.to_csv("data/preprocessed_data/df_female.csv")
df_FIM_total.to_csv("data/preprocessed_data/df_FIM_total.csv")
df_acute_days.to_csv("data/preprocessed_data/df_acute_days.csv")
df_brain_injury.to_csv("data/preprocessed_data/df_brain_injury.csv")
df_outcome.to_csv("data/preprocessed_data/df_outcome.csv")

df_admissions_resampled.to_csv("data/preprocessed_data/df_admissions_resampled.csv")